In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 18.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.1 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446661 sha256=af1f4601fb5ef2b8106a15f68d5c2032d6bbdaf52820a20f5160dfad124d4686
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103

In [ ]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [ ]:
env_id = "PandaReachDense-v3"

env = gym.make(env_id)

s_size = env.observation_space.shape
a_size = env.action_space

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('___Observation Space___\n')
print(f'The State Space is: {s_size}')
print(f'Sample observation {env.observation_space.sample()}')

___Observation Space___

The State Space is: None
Sample observation OrderedDict([('achieved_goal', array([ 9.97423 , -8.717325, -8.804689], dtype=float32)), ('desired_goal', array([-0.49644962,  8.171489  , -0.67235917], dtype=float32)), ('observation', array([ 9.077205  , -8.264178  , -0.66076607, -1.771498  , -2.380359  ,
       -2.9072845 ], dtype=float32))])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print('___Action Space___\n')
print(f'The Action Space is: {a_size}')
print(f'Action Space Sample {env.action_space.sample()}')

___Action Space___

The Action Space is: Box(-1.0, 1.0, (3,), float32)
Action Space Sample [ 0.03498756  0.54859185 -0.01483065]


In [ ]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

In [ ]:
model = A2C(policy='MultiInputPolicy', env=env, verbose=1)

Using cuda device


In [ ]:
 model.learn(1_000_000)

Streaming output truncated to the last 5000 lines.
|    std                | 0.352    |
|    value_loss         | 0.000117 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.86     |
|    ep_rew_mean        | -0.228   |
|    success_rate       | 1        |
| time/                 |          |
|    fps                | 349      |
|    iterations         | 23800    |
|    time_elapsed       | 1360     |
|    total_timesteps    | 476000   |
| train/                |          |
|    entropy_loss       | -0.98    |
|    explained_variance | 0.958    |
|    learning_rate      | 0.0007   |
|    n_updates          | 23799    |
|    policy_loss        | -0.00583 |
|    std                | 0.35     |
|    value_loss         | 0.000139 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.88     |
|    ep_rew_mean        

In [ ]:
model.save('a2c-PandaReachDense-v3')
env.save('vec_normalize.pkl')

In [ ]:
eval_env = DummyVecEnv([lambda: gym.make('PandaReachDense-v3')])
eval_env = VecNormalize.load('vec_normalize.pkl', eval_env)

eval_env.render_mode = 'rgb_array'

eval_env.training = False

eval_env.norm_reward = False

model = A2C.load('a2c-PandaReachDense-v3')

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f'Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}')

Mean reward = -0.21 +/- 0.10


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [ ]:
notebook_login()
!git config --global credential.helper store

In [ ]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"khadivi-ah/a2c-{env_id}",
    commit_message="Initial commit",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

policy.optimizer.pth:   0%|          | 0.00/48.5k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

a2c-PandaReachDense-v3.zip:   0%|          | 0.00/114k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/khadivi-ah/a2c-PandaReachDense-v3/commit/712a80cb3108330d40d7cb296c9a3c4cb744418c', commit_message='Initial commit', commit_description='', oid='712a80cb3108330d40d7cb296c9a3c4cb744418c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
env_id = 'PandaPickAndPlace-v3'
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

model = A2C(policy='MultiInputPolicy', env=env, verbose=1)

Using cuda device


In [ ]:
model.learn(1_000_000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
|    std                | 0.766    |
|    value_loss         | 3.1e-05  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49.5     |
|    ep_rew_mean        | -49.5    |
|    success_rate       | 0.01     |
| time/                 |          |
|    fps                | 340      |
|    iterations         | 23800    |
|    time_elapsed       | 1398     |
|    total_timesteps    | 476000   |
| train/                |          |
|    entropy_loss       | -4.61    |
|    explained_variance | 0.916    |
|    learning_rate      | 0.0007   |
|    n_updates          | 23799    |
|    policy_loss        | 0.0104   |
|    std                | 0.768    |
|    value_loss         | 2.65e-05 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 47.5     |
|    ep_rew_mean        

In [ ]:
model.save('a2c-PandaPickAndPlace-v3')
env.save('vec_normalize.pk')

In [ ]:
eval_env = DummyVecEnv([lambda: gym.make('PandaPickAndPlace-v3')])
eval_env = VecNormalize.load('vec_normalize.pk', eval_env)

eval_env.training = False
eval_env.nrom_reward = False

model = A2C.load('a2c-PandaPickAndPlace-v3')

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f'Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -3.53 +/- 1.77


In [ ]:
notebook_login()

In [ ]:
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"khadivi-ah/a2c-{env_id}",
    commit_message="Initial commit",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpb3_6ff1t/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpb3_6ff1t/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpb3_6ff1t/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpb3_6ff1t/-step-0-to-step-1000.mp4
ℹ Pushing repo khadivi-ah/a2c-PandaPickAndPlace-v3 to the Hugging Face
Hub


...

a2c-PandaPickAndPlace-v3.zip:   0%|          | 0.00/130k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

vec_normalize.pkl:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/khadivi-ah/a2c-PandaPickAndPlace-v3/tree/main/


CommitInfo(commit_url='https://huggingface.co/khadivi-ah/a2c-PandaPickAndPlace-v3/commit/8ff10b362cb61a5efaa84ace155e72630235b892', commit_message='Initial commit', commit_description='', oid='8ff10b362cb61a5efaa84ace155e72630235b892', pr_url=None, pr_revision=None, pr_num=None)